# DSCI 100 Group Project

### Introduction
To do: 
- Write a bit about what the data set is (couple sentences about PLAICraft, purpose of the study)
- State the question: demand forecasting
- Describe the dataset used: sessions AND players

As is, the question is kind of too broad, so here are some easier ones to run by the TA:
- Mean number of sessions in each hour of the day: pick the top x number of hours as "peak hours" -- can use bootstrapping for this
- 

### Exploratory Data Analysis

To do: 
- Split the data into a training set and a testing set. Only explore the training set.
- Calculate basic summary statistics on time played and make a few visualizations, like in planning document